# RUL estimation UNIBO Powertools Dataset

In [1]:
import numpy as np
import pandas as pd
import scipy.io
import math
import os
import ntpath
import sys
import logging
import time
import sys
import random

from importlib import reload
import plotly.graph_objects as go

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.models import Model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, Masking


IS_COLAB = False
IS_TRAINING = True
RESULT_NAME = ""

if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    data_path = "/content/drive/My Drive/battery-state-estimation/battery-state-estimation/"
else:
    data_path = "../../"

sys.path.append(data_path)
from data_processing.unibo_powertools_data import UniboPowertoolsData, CycleCols
from data_processing.model_data_handler import ModelDataHandler
from data_processing.prepare_rul_data import RulHandler

### Config logging

In [2]:
reload(logging)
logging.basicConfig(format='%(asctime)s [%(levelname)s]: %(message)s', level=logging.DEBUG, datefmt='%Y/%m/%d %H:%M:%S')

# Load Data

In [3]:
dataset = UniboPowertoolsData(
    test_types=[],
    chunk_size=1000000,
    lines=[37, 40],
    charge_line=37,
    discharge_line=40,
    base_path=data_path
)

2021/07/28 16:25:39 [DEBUG]: Start loading data with lines: [37, 40], types: [] and chunksize: 1000000...
2021/07/28 16:26:18 [DEBUG]: Finish loading data.
2021/07/28 16:26:18 [INFO]: Loaded raw dataset A data with cycle row count: 15384908 and capacity row count: 39585
2021/07/28 16:26:18 [DEBUG]: Start cleaning cycle raw data...
2021/07/28 16:26:33 [DEBUG]: Finish cleaning cycle raw data.
2021/07/28 16:26:33 [INFO]: Removed 11 rows of abnormal cycle raw data.
2021/07/28 16:26:33 [DEBUG]: Start cleaning capacity raw data...
2021/07/28 16:26:33 [DEBUG]: Finish cleaning capacity raw data.
2021/07/28 16:26:33 [INFO]: Removed 1 rows of abnormal capacity raw data.
2021/07/28 16:26:33 [DEBUG]: Start assigning charging raw data...
2021/07/28 16:26:34 [DEBUG]: Finish assigning charging raw data.
2021/07/28 16:26:34 [INFO]: [Charging] cycle raw count: 12160812, capacity raw count: 19800
2021/07/28 16:26:34 [DEBUG]: Start assigning discharging raw data...
2021/07/28 16:26:35 [DEBUG]: Finish ass

In [4]:
train_names = [
    '000-DM-3.0-4019-S',#minimum capacity 1.48
    '001-DM-3.0-4019-S',#minimum capacity 1.81
    '002-DM-3.0-4019-S',#minimum capacity 2.06

    '009-DM-3.0-4019-H',#minimum capacity 1.41
    '010-DM-3.0-4019-H',#minimum capacity 1.44

    '014-DM-3.0-4019-P',#minimum capacity 1.7
    '015-DM-3.0-4019-P',#minimum capacity 1.76
    '016-DM-3.0-4019-P',#minimum capacity 1.56
    '017-DM-3.0-4019-P',#minimum capacity 1.29
    #'047-DM-3.0-4019-P',#new 1.98
    #'049-DM-3.0-4019-P',#new 2.19



    '007-EE-2.85-0820-S',#2.5
    '008-EE-2.85-0820-S',#2.49
    '042-EE-2.85-0820-S',#2.51

    '043-EE-2.85-0820-H',#2.31


    '040-DM-4.00-2320-S',#minimum capacity 3.75, cycles 188


    '018-DP-2.00-1320-S',#minimum capacity 1.82
    #'019-DP-2.00-1320-S',#minimum capacity 1.61
    '036-DP-2.00-1720-S',#minimum capacity 1.91
    '037-DP-2.00-1720-S',#minimum capacity 1.84
    '038-DP-2.00-2420-S',#minimum capacity 1.854 (to 0)
    '050-DP-2.00-4020-S',#new 1.81
    '051-DP-2.00-4020-S',#new 1.866    
    
]

test_names = [
    '003-DM-3.0-4019-S',#minimum capacity 1.84

    '011-DM-3.0-4019-H',#minimum capacity 1.36

    '013-DM-3.0-4019-P',#minimum capacity 1.6



    '006-EE-2.85-0820-S',# 2.621
    
    '044-EE-2.85-0820-H',# 2.43



    '039-DP-2.00-2420-S',#minimum capacity 1.93



    '041-DM-4.00-2320-S',#minimum capacity 3.76, cycles 190
]

In [5]:
dataset.prepare_data(train_names, test_names)
dataset_handler = ModelDataHandler(dataset, [
    CycleCols.VOLTAGE,
    CycleCols.CURRENT,
    CycleCols.TEMPERATURE
])

rul_handler = RulHandler()

2021/07/28 16:26:35 [DEBUG]: Start preparing data for training: ['000-DM-3.0-4019-S', '001-DM-3.0-4019-S'] and testing: ['003-DM-3.0-4019-S']...
../../data_processing/unibo_powertools_data.py:273: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  cyc_data = np.array(cyc_data)
../../data_processing/unibo_powertools_data.py:273: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  cyc_data = np.array(cyc_data)
2021/07/28 16:26:44 [DEBUG]: Finish getting training and testing charge data.
../../data_processing/unibo_powertools_data.py:273: VisibleDeprecationWarning: 

# Data preparation

In [6]:
CAPACITY_THRESHOLDS = {
  3.0 : 2.7,#th 90% - min 2.1, 70%
  2.85 : 2.7,#th 94.7% - min 2.622, 92%
  2.0 : 1.93,#th 96.5% - min 1.93, 96.5%
  4.0 : 3.77,#th 94.2% - min 3.77 94.2%
  4.9 : 4.7,#th 95.9% - min 4.3, 87.7%
  5.0 : 4.5#th 90% - min 3.63, 72.6%
}
N_CYCLE = 500
WARMUP_TRAIN = 15
WARMUP_TEST = 30

(train_x, train_y_soh, test_x, test_y_soh,
  train_battery_range, test_battery_range,
  time_train, time_test, current_train, current_test) = dataset_handler.get_discharge_whole_cycle_future(train_names, test_names)

train_x = train_x[:,:284,:]
test_x = test_x[:,:284,:]
print("cut train shape {}".format(train_x.shape))
print("cut test shape {}".format(test_x.shape))

x_norm = rul_handler.Normalization()
train_x, test_x = x_norm.normalize(train_x, test_x)


train_y = rul_handler.prepare_y_future(train_names, train_battery_range, train_y_soh, current_train, time_train, CAPACITY_THRESHOLDS)
del globals()["current_train"]
del globals()["time_train"]
test_y = rul_handler.prepare_y_future(test_names, test_battery_range, test_y_soh, current_test, time_test, CAPACITY_THRESHOLDS)
del globals()["current_test"]
del globals()["time_test"]

../../data_processing/model_data_handler.py:91: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x = np.array(
2021/07/28 16:26:50 [INFO]: Train x: (721, 214, 3), train y soh: (721, 1) | Test x: (356, 214, 3), test y soh: (356, 1) | 
                            battery n cycle train: (2,), battery n cycle test: (1,), 
                            time train: (721, 214, 1), time test: (356, 214, 1) |
                            raw current train: (721, 214), raw current test: (356, 214) |
                            
2021/07/28 16:26:50 [INFO]: battery step: [353 721]
2021/07/28 16:26:50 [INFO]: battery ranges: [75542, 154294]
2021/07/28 16:26:50 [INFO]: processing range 0 - 75542
2021/07/28 16:26:50 [INFO]: processing range 75542 - 154294
2021/07/28 16:26:50 [INFO]: Train 

cut train shape (721, 214, 3)
cut test shape (356, 214, 3)


## compressing x using autoencoder

In [7]:
AUTOENCODER_WEIGHTS = '2021-07-19-17-29-18_autoencoder_unibo_powertools'

# Model definition
opt = tf.keras.optimizers.Adam(learning_rate=0.0002)
LATENT_DIM = 10

class Autoencoder(Model):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=(train_x.shape[1], train_x.shape[2])),
            #layers.MaxPooling1D(5, padding='same'),
            layers.Conv1D(filters=16, kernel_size=5, strides=2, activation='relu', padding='same'),
            layers.Conv1D(filters=8, kernel_size=3, strides=2, activation='relu', padding='same'),
            layers.Flatten(),
            layers.Dense(self.latent_dim, activation='relu')
        ])
        self.decoder = tf.keras.Sequential([
            layers.Input(shape=(self.latent_dim)),
            layers.Dense(568, activation='relu'),
            layers.Reshape((71, 8)),
            layers.Conv1DTranspose(filters=8, kernel_size=3, strides=2, activation='relu', padding='same'),
            layers.Conv1DTranspose(filters=16, kernel_size=5, strides=2, activation='relu', padding='same'),
            layers.Conv1D(3, kernel_size=3, activation='relu', padding='same'),
            #layers.UpSampling1D(5),
        ])


    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Autoencoder(LATENT_DIM)
autoencoder.compile(optimizer=opt, loss='mse', metrics=['mse', 'mae', 'mape', tf.keras.metrics.RootMeanSquaredError(name='rmse')])
autoencoder.encoder.summary()
autoencoder.decoder.summary()

autoencoder.load_weights(data_path + 'results/trained_model/%s/model' % AUTOENCODER_WEIGHTS)


# compression
train_x = autoencoder.encoder(train_x).numpy()
test_x = autoencoder.encoder(test_x).numpy()
print("compressed train x shape {}".format(train_x.shape))
print("compressed test x shape {}".format(test_x.shape))
train_x = train_x[:,~np.all(train_x == 0, axis=0)]
test_x = test_x[:,~np.all(test_x == 0, axis=0)]
print("compressed test x shape without zero column {}".format(test_x.shape))
print("compressed test x shape without zero column {}".format(test_x.shape))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 107, 16)           256       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 54, 8)             392       
_________________________________________________________________
flatten (Flatten)            (None, 432)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                4330      
Total params: 4,978
Trainable params: 4,978
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 568)               6248      
________________________________

ValueError: Tensor's shape (568, 10) is not compatible with supplied shape (432, 10)

In [ ]:
x_norm = rul_handler.Normalization()
train_x, test_x = x_norm.normalize(train_x, test_x)
train_x = rul_handler.battery_life_to_time_series(train_x, N_CYCLE, train_battery_range)
test_x = rul_handler.battery_life_to_time_series(test_x, N_CYCLE, test_battery_range)

train_x, train_y, train_battery_range, train_y_soh = rul_handler.delete_initial(train_x, train_y, train_battery_range, train_y_soh, WARMUP_TRAIN)
test_x, test_y, test_battery_range, test_y_soh = rul_handler.delete_initial(test_x, test_y, test_battery_range, test_y_soh, WARMUP_TEST)

# first one is SOH, we keep only RUL
train_y = train_y[:,1]
test_y = test_y[:,1]

## Y normalization

In [ ]:
y_norm = rul_handler.Normalization()
train_y, test_y = y_norm.normalize(train_y, test_y)

# Model training

In [ ]:
if IS_TRAINING:
    EXPERIMENT = "lstm_autoencoder_rul_unibo_powertools"

    experiment_name = time.strftime("%Y-%m-%d-%H-%M-%S") + '_' + EXPERIMENT
    print(experiment_name)

    # Model definition

    opt = tf.keras.optimizers.Adam(lr=0.000003)

    model = Sequential()
    model.add(Masking(input_shape=(train_x.shape[1], train_x.shape[2])))
    model.add(LSTM(128, activation='selu',
                    return_sequences=True,
                    kernel_regularizer=regularizers.l2(0.0002)))
    model.add(LSTM(64, activation='selu', return_sequences=False,
                    kernel_regularizer=regularizers.l2(0.0002)))
    model.add(Dense(64, activation='selu', kernel_regularizer=regularizers.l2(0.0002)))
    model.add(Dense(32, activation='selu', kernel_regularizer=regularizers.l2(0.0002)))
    model.add(Dense(1, activation='linear'))
    model.summary()

    model.compile(optimizer=opt, loss='huber', metrics=['mse', 'mae', 'mape', tf.keras.metrics.RootMeanSquaredError(name='rmse')])

In [ ]:
if IS_TRAINING:
    history = model.fit(train_x, train_y, 
                                epochs=500, 
                                batch_size=32, 
                                verbose=1,
                                validation_split=0
                               )

In [ ]:
if IS_TRAINING:
    model.save(data_path + 'results/trained_model/%s.h5' % experiment_name)

    hist_df = pd.DataFrame(history.history)
    hist_csv_file = data_path + 'results/trained_model/%s_history.csv' % experiment_name
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)
    history = history.history

In [ ]:
if not IS_TRAINING:
    history = pd.read_csv(data_path + 'results/trained_model/%s_history.csv' % RESULT_NAME)
    model = keras.models.load_model(data_path + 'results/trained_model/%s.h5' % RESULT_NAME)
    model.summary()

In [ ]:
if not IS_TRAINING:
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(history)

### Testing

In [ ]:
results = model.evaluate(test_x, test_y, return_dict = True)
print(results)
max_rmse = 0
for index in range(test_x.shape[0]):
    result = model.evaluate(np.array([test_x[index, :, :]]), np.array([test_y[index]]), return_dict = True, verbose=0)
    max_rmse = max(max_rmse, result['rmse'])
print("Max rmse: {}".format(max_rmse))

# Results Visualization

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=history['loss'],
                    mode='lines', name='train'))
fig.update_layout(title='Loss trend',
                  xaxis_title='epoch',
                  yaxis_title='loss',
                  width=1400,
                  height=600)
fig.show()

In [ ]:
train_predictions = model.predict(train_x)

train_y = y_norm.denormalize(train_y)
train_predictions = y_norm.denormalize(train_predictions)

In [ ]:
a = 0
for b in train_battery_range:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train_y_soh[a:b], y=train_predictions[a:b,0],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(x=train_y_soh[a:b], y=train_y[a:b],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on training',
                    xaxis_title='SoH Capacity',
                    yaxis_title='Remaining Ah until EOL',
                    xaxis={'autorange':'reversed'},
                    width=1400,
                    height=600)
    fig.show()
    a = b

In [ ]:
a = 0
for b in train_battery_range:
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=train_predictions[a:b,0],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(y=train_y[a:b],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on training',
                    xaxis_title='Cycle',
                    yaxis_title='Remaining Ah until EOL',
                    width=1400,
                    height=600)
    fig.show()
    a = b

In [ ]:
test_predictions = model.predict(test_x)

test_y = y_norm.denormalize(test_y)
test_predictions = y_norm.denormalize(test_predictions)

In [ ]:
a = 0
for b in test_battery_range:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=test_y_soh[a:b], y=test_predictions[a:b,0],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(x = test_y_soh[a:b], y=test_y[a:b],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on testing',
                    xaxis_title='SoH Capacity',
                    yaxis_title='Remaining Ah until EOL',
                    xaxis={'autorange':'reversed'},
                    width=1400,
                    height=600)
    fig.show()
    a = b

In [ ]:
a = 0
for b in test_battery_range:
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=test_predictions[a:b, 0],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(y=test_y[a:b],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on testing',
                    xaxis_title='Cycle',
                    yaxis_title='Remaining Ah until EOL',
                    width=1400,
                    height=600)
    fig.show()
    a = b